In [1]:
!pip install beautifulsoup4 # permite levantar html

In [2]:
from gensim.models import Word2Vec

In [3]:
class EpochSaver: pass

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
#DATA_HOME = '../../data/w2v'
#DATA_HOME = '../../../machine_learning_practico/data/w2v/'
DATA_HOME=r"/content/gdrive/MyDrive/ml-practico/data/w2v/"
!mkdir -p $DATA_HOME

In [ ]:
!ls $DATA_HOME # revisamos que este vacio

In [ ]:
# ya esta entrenado el word2vec, lo levanto
# lo entrene yo

#!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10 -O $DATA_HOME/epoch_10
#!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.trainables.syn1neg.npy -O $DATA_HOME/epoch_10.trainables.syn1neg.npy
#!wget https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.wv.vectors.npy -O $DATA_HOME/epoch_10.wv.vectors.npy

--2020-11-26 13:45:48--  https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10
Resolving machine-learning-practico.s3.amazonaws.com (machine-learning-practico.s3.amazonaws.com)... 52.217.9.164
Connecting to machine-learning-practico.s3.amazonaws.com (machine-learning-practico.s3.amazonaws.com)|52.217.9.164|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-11-26 13:45:48 ERROR 404: Not Found.

--2020-11-26 13:45:48--  https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.trainables.syn1neg.npy
Resolving machine-learning-practico.s3.amazonaws.com (machine-learning-practico.s3.amazonaws.com)... 52.217.9.164
Connecting to machine-learning-practico.s3.amazonaws.com (machine-learning-practico.s3.amazonaws.com)|52.217.9.164|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-11-26 13:45:48 ERROR 404: Not Found.

--2020-11-26 13:45:48--  https://machine-learning-practico.s3.amazonaws.com/w2v/epoch_10.wv.vectors.npy
Resol

In [6]:
from pathlib import Path
DATA_HOME = Path(DATA_HOME)

In [7]:
model = Word2Vec.load(str(DATA_HOME / 'epoch_10'))

In [8]:
model.wv.vectors.shape #854868*100 es el numero de parametros, 100 es el numero de dimensiones
#model.wv.syn0.shape, 854868 es el numero de personas/actores que le calculo un vector

(854868, 100)

**con aprendizaje no supervisado es muy dificil identificar overfitting. si algo lo vi en pocos contextos, eso por ahi lo tengo que sacar. Tiene que ver con particularidades del dataset. La complejidad del modelo es altisima (muchos parametros). Una forma es en vez de elegir 100 dimensiones, elijo 30, asi tengo menos parametros.**

In [9]:
!pip install requests_cache

In [10]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import requests
import requests_cache

requests_cache.install_cache('imdb')
#entra a la pagina y te busca la foto de la actriz
def get_name(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    return soup.select('.header .itemprop')[0].text

def get_image(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    candidates = soup.select('#name-poster')
    return candidates[0].attrs['src'] if candidates else 'https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB466677935_.png'

def render_person(id):
    name = get_name(id)
    picture = get_image(id)
    return f"""
    <div style="width: 150px; text-align: center">
        <h4 style='margin-top: -5px'>{name}</h4>
        <div style='font-size:75%; margin-bottom: 5px'>{id}</div>
        <a href="https://www.imdb.com/name/{id}" target="_blank">
            <img style="width: 100px; display: block; margin-left: auto; margin-right: auto;" src="{picture}"/>
        </a>
    </div>
    """

In [11]:
display(HTML(render_person('nm0001401'))) # si clickeo la foto me manda a la pagina de angelina en imdb

In [12]:
# me da los parecidos a angelina jolie
def show_similars(id, n=10):
    display(HTML(render_person(id)))
    renders = []
    for similar_id, score in model.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
    )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

In [13]:
show_similars('nm0001401') # esta bien, son todos de hollywood

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
show_similars('nm0201857') # rendereame los parecidos a darin

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
show_similars('nm0201857')
#rendereame a ricardo darin

In [ ]:
# ernesto sabato puede ser overfitting
model.wv.vocab['nm0844531'].count
# en 17 pelis aparece ernesto sabato en el training set. no alcanza para formar un buen vector
# con 17 peliculas determino los 100 floats del vector de sabato

17

In [14]:
model.wv.vocab['nm0201857'].count
#darin aparecio 875 veces 

875

In [ ]:
show_similars('nm0863906') # son todos argentinos

In [ ]:
show_similars('nm0655585')

**tal vez la gente que aparecio menos de 20 veces hay que volarla.** 

In [ ]:
show_similars('nm0000375')

In [ ]:
show_similars('nm0511088')

In [ ]:
show_similars('nm0000150')

quiero ver si el word2vec anda bien o mal. Lo malo es que metimos en el mismo word2vec actores, gente de sonido. Eso me mete ruido, me linkea a actores que actuaron con el mismo productor de sonido pero que no tienen nada que ver. Habria que hacer un word2vec solo para actores. 

el modelo despues va a usar como features estos vectores de word2vec. La representacion es dime con  quien andas y te dire quien eres.

# Algunos random

In [ ]:
#que onda si elijo gente random
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

from lib import data

In [ ]:
from pathlib import Path
PATH = Path('../../data/')

In [ ]:
import pandas as pd

title_basics = data.load_title_basics(PATH)
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')
ratings_df = pd.read_csv(PATH / 'title.ratings.tsv', sep='\t')

/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
candidates = (
    principals_df[
        principals_df.category.isin(['actress', 'actor', 'director']) &
        principals_df.tconst.isin(set(title_basics[title_basics.startYear >= 1980].tconst)) &
        principals_df.tconst.isin(set(ratings_df[ratings_df.numVotes > 100000].tconst))
    ].nconst.unique()
)

candidates = [c for c in candidates if c in model.wv.vocab]
len(candidates)

In [ ]:
from random import choice

for _ in range(10):
    id = choice(candidates)
    show_similars(id)

entreno word2vec, entreno el modelo y desps me fijo en el validation set que tal da. Probar varios word2vec: ahi tengo varios hiperparametros (epochs, dimensiones del word2vec). 

In [17]:
model.wv.vectors[0] # este es el vector y no tiene mucho sentido, pero las distancias si parecen tener algo de sentido


array([ 3.5759606 ,  0.54323846, -1.6441029 , -0.37274   ,  3.2667105 ,
       -6.814562  ,  0.8697774 ,  2.3411584 ,  0.3218174 ,  3.391918  ,
       -0.6219553 ,  4.572124  , -1.6139559 ,  1.33083   ,  3.7648106 ,
        6.383616  ,  1.8172505 ,  1.199131  ,  2.0477593 , -0.52280235,
        1.0283022 , -3.2430482 ,  4.0712223 , -5.501521  ,  2.2529712 ,
        4.0140705 ,  0.21022673,  0.14982906, -1.6049697 ,  2.832901  ,
       -4.3372927 , -5.7438507 , -2.573742  ,  2.423227  , -0.08488566,
        1.869348  , -0.55482846,  1.3407089 ,  2.8266034 , -0.85136306,
       -3.9748695 ,  1.2272302 , -2.3914018 , -2.3496413 ,  2.2400901 ,
        1.2617271 ,  4.438213  ,  1.381446  , -0.133726  ,  4.0730023 ,
       -6.859824  ,  3.5540445 ,  2.351341  ,  3.1067836 ,  2.92614   ,
       -4.242716  , -4.9822793 ,  1.9240373 ,  1.0990676 ,  1.03669   ,
       -2.2562568 , -3.6222389 ,  4.3597527 ,  2.9515462 , -0.8173002 ,
        2.9032147 , -3.0934308 , -5.2200174 , -1.1145204 ,  0.07

In [18]:
render_person(model.wv.index2word[0]) # esto tarda
model.wv.index2word[0]

KeyboardInterrupt: ignored